In [1]:
from defusedxml import ElementTree as DET
import xml.etree.ElementTree as ET
import pandas as pd
import httpx
import uuid

In [2]:
r = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/chargeList.xml')

tree = ET.ElementTree(DET.fromstring(r.text)).getroot()
data = []

for element in tree:
    code = element.find("specCode").text
    article = element.find("specArticle").text
    definition = element.find("specDefinition").text

    data.append({"code": code, "article": article, "definition": definition})

articles: pd.DataFrame = pd.DataFrame(data)
articles

,code,article,definition
0,081-A-,81,"Conspiracy; war crime, lead to death"
1,082---,82,Solicitation of an offense
2,082-B1,82,Solicit desertion
3,082-B2,82,Solicit mutiny
4,082-B3,82,Solicit misbehavior before enemy
...,...,...,...
766,134-Z2,134,Other offenses under Article 134
767,134R6A,134,Child pornography; possession
768,134R6B,134,Child pornography; intent to distribute
769,134R6C,134,Child pornography; distribution


In [9]:
r = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/baseList.xml')

tree = ET.ElementTree(DET.fromstring(r.text)).getroot()
data  = []

for element in tree:
    base = element.find("base").text
    base_name = element.find("baseName").text
    state_abv = element.find("stateAbbrev").text
    state_long = element.find("stateLongName").text
    phone = element.find("basePhone").text

    data.append(
        {
            "base": base,
            "base_name": base_name,
            "state_abv": state_abv,
            "state": state_long,
            "phone_number": phone,
        }
    )
bases = pd.DataFrame(data)
bases

,base,base_name,state_abv,state,phone_number
0,None,UNDEFINED,XX,Undefined Location,000-000-0000
1,ATU2,USAFCENT - 466 AEG,AE,Deployed Location,803-895-2875
2,ATUH,USAFCENT - 455 AEW,AE,Deployed Location,803-895-2875
3,FBKA,USAFCENT - 451 AEW,AE,Deployed Location,803-895-2875
4,FDMY,USAFCENT - 376 AEW,AE,Deployed Location,803-895-2875
...,...,...,...,...,...
106,TRC1,Vandenberg SFB,CA,California,805-606-5395
107,TRC2,Peterson-Schreiver Garrison,CO,Colorado,719-556-5815
108,TRC3,Peterson-Schreiver Garrison,CO,Colorado,719-556-5815
109,TRC4,Vandenberg SFB,CA,California,805-606-5395


In [12]:
r = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/caseList.1.7.xml')

root = ET.fromstring(r.text)

element_data = []
sub_element_data = []


# loop through each element, and sub element and parse data
for tree in root:
    for element in tree:
        # Generate a uniue id for each element
        id: str = str(uuid.uuid1())

        element.attrib["fk_id"] = id

        case_info: dict = {element.tag: element.attrib}

        for sub_element in element:
            sub_element.attrib["case_id"] = id

            sub_element_data.append(sub_element.attrib)

        element_data.append(case_info)

case_charges = []
case_personnel = []

# Loop through the sub elements and parse data

for values in sub_element_data:
    if "code" in values:
        case_charge_data = {
            "case_id": values.get("case_id"),
            "code": values.get("code"),
            "prefix": values.get("prefix"),
        }

        case_charges.append(case_charge_data)

    else:
        case_personnel_data = {
            "case_id": values.get("case_id"),
            "order": values.get("order"),
            "title": values.get("title"),
            "name": values.get("name"),
        }

        case_personnel.append(case_personnel_data)

df_case_info = pd.json_normalize(element_data)
df_case_charges = pd.DataFrame(case_charges)
df_case_personnel = pd.DataFrame(case_personnel)


In [13]:
df_case_info

,caseInfo.rankAbbrv,caseInfo.rank,caseInfo.rankOrder,caseInfo.baseCode,caseInfo.nameFirst,caseInfo.nameLast,caseInfo.caseType,caseInfo.scheduledTrialStartDate,caseInfo.scheduledTrialEndDate,caseInfo.estimatedTrialDays,caseInfo.fk_id,caseInfo.tr140aUrl,caseInfo.trialCompletionDate,caseInfo.verdict,caseInfo.forum,caseInfo.sentence,state.abbrv,state.name,state.fk_id
0,SSgt,Staff Sergeant,5,FTFA,M,PHIPPS,Special Court Martial,2024-12-03,2024-12-06,3,7649f2ee-c0d9-11ef-9b02-cafaa44913ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AB,Airman Basic,1,MSET,C,MATOS,Special Court Martial,2024-12-05,2024-12-06,1,7649f4c4-c0d9-11ef-9b02-cafaa44913ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SrA,Senior Airman,4,LJYC,D,EDWARDS,Special Court Martial,2024-12-09,2024-12-14,5,7649f50a-c0d9-11ef-9b02-cafaa44913ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MSgt,Master Sergeant,7,GHLN,N,GONZALEZ,Special Court Martial,2024-12-09,2024-12-13,4,7649f53c-c0d9-11ef-9b02-cafaa44913ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SMSgt,Senior Master Sergeant,8,FSPM,D,LAMM,Special Court Martial,2024-12-09,2024-12-14,5,7649f55a-c0d9-11ef-9b02-cafaa44913ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WA,Washington,764b2628-c0d9-11ef-9b02-cafaa44913ac
2639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI,Wisconsin,764b263c-c0d9-11ef-9b02-cafaa44913ac
2640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WV,West Virginia,764b2650-c0d9-11ef-9b02-cafaa44913ac
2641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WY,Wyoming,764b2664-c0d9-11ef-9b02-cafaa44913ac


In [14]:
df_case_charges

,case_id,code,prefix
0,7649f2ee-c0d9-11ef-9b02-cafaa44913ac,112AC1,
1,7649f4c4-c0d9-11ef-9b02-cafaa44913ac,087AA2,
2,7649f4c4-c0d9-11ef-9b02-cafaa44913ac,091-C2,
3,7649f4c4-c0d9-11ef-9b02-cafaa44913ac,092-B-,
4,7649f4c4-c0d9-11ef-9b02-cafaa44913ac,112AC4,
...,...,...,...
4516,764b1f0c-c0d9-11ef-9b02-cafaa44913ac,112AC1,
4517,764b1f0c-c0d9-11ef-9b02-cafaa44913ac,112AD1,
4518,764b1f0c-c0d9-11ef-9b02-cafaa44913ac,118-A-,Conspiracy to commit -
4519,764b1f0c-c0d9-11ef-9b02-cafaa44913ac,126-A-,


In [15]:
df_case_personnel

,case_id,order,title,name
0,7649f2ee-c0d9-11ef-9b02-cafaa44913ac,1,Military Judge:,Col Joshua Daniel ROSEN
1,7649f2ee-c0d9-11ef-9b02-cafaa44913ac,2,Circuit Trial Counsel:,
2,7649f2ee-c0d9-11ef-9b02-cafaa44913ac,3,Trial Counsel:,"1 Lt Clayton B. STONE, Capt Joshua Alexander LEE"
3,7649f2ee-c0d9-11ef-9b02-cafaa44913ac,4,Military Defense Counsel:,"Capt Matthew Randolph FIFE, Capt Rachel A. LAMM"
4,7649f4c4-c0d9-11ef-9b02-cafaa44913ac,1,Military Judge:,
...,...,...,...,...
12769,764b1f0c-c0d9-11ef-9b02-cafaa44913ac,1,Military Judge:,"Col Vance Spath, Lt Col Shelly Schools"
12770,764b1f0c-c0d9-11ef-9b02-cafaa44913ac,2,Circuit Trial Counsel:,
12771,764b1f0c-c0d9-11ef-9b02-cafaa44913ac,3,Trial Counsel:,"Capt (E) MARY J. WOOD, Capt MATTHEW R. HRACHO"
12772,764b1f0c-c0d9-11ef-9b02-cafaa44913ac,4,Assistant Trial Counsel:,Capt TYLER J. KAIDO


In [16]:
r = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/caseList.1.7.xml')

root = ET.fromstring(r.text)

In [26]:
charge_list_response = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/chargeList.xml')

charge_list_tree = ET.ElementTree(DET.fromstring(charge_list_response.text)).getroot()

In [31]:
case_list_response = httpx.get('https://legalassistance.law.af.mil/AMJAMS/PublicDocket/caseList.1.7.xml')

case_list_tree = ET.ElementTree(DET.fromstring(case_list_response.text)).getroot()

In [32]:
for child in case_list_tree:
    print(child.tag, child.attrib)

cases {}
trialResults {}
states {}


In [34]:
print(case_list_response.text)

<root>
<cases>
<caseInfo rankAbbrv="SSgt" rank="Staff Sergeant" rankOrder= "5" baseCode="FTFA" nameFirst="M" nameLast="PHIPPS" caseType="Special Court Martial" scheduledTrialStartDate="2024-12-03" scheduledTrialEndDate="2024-12-06" estimatedTrialDays="3">
<charge code="112AC1" prefix=""/>
<trialPersonnel order="1" title="Military Judge: " name="Col Joshua Daniel ROSEN"/>
<trialPersonnel order="2" title="Circuit Trial Counsel: " name=""/>
<trialPersonnel order="3" title="Trial Counsel: " name="1 Lt Clayton B. STONE, Capt Joshua Alexander LEE"/>
<trialPersonnel order="4" title="Military Defense Counsel: " name="Capt Matthew Randolph FIFE, Capt Rachel A. LAMM"/>
</caseInfo>
<caseInfo rankAbbrv="AB" rank="Airman Basic" rankOrder= "1" baseCode="MSET" nameFirst="C" nameLast="MATOS" caseType="Special Court Martial" scheduledTrialStartDate="2024-12-05" scheduledTrialEndDate="2024-12-06" estimatedTrialDays="1">
<charge code="087AA2" prefix=""/>
<charge code="091-C2" prefix=""/>
<charge code="09